In [1]:
!pip install feedparser
!pip install newspaper3k
!pip install konlpy
!pip install lxml[html_clean]

import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=cfc70ed63edac174a104cd8d0a2fb282fc522a6af99858228ea3abbff765246d
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.0 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=f0ffd8fb21a4c17c91a1a1b97c367a66d84c780144e70fbcf6a959c74b22e8f4
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df4

## 단계 1: 모든 RSS파일(xml형식)을 돌아다니면서 기사의 제목/link를 추출

In [2]:
urls = [
    'https://rss.etnews.com/Section901.xml',
    'https://rss.etnews.com/Section902.xml',
    'https://rss.etnews.com/Section903.xml',
    'https://rss.etnews.com/Section904.xml'
]

In [3]:
def crawl_rss(urls):
  array_rss = []
  set_titles = set()
  for url in urls:
    print('[Crawl URL]', url)
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries:
      if p.title not in set_titles:
        array_rss.append({'title':p.title, 'link':p.link})
        set_titles.add(p.title)
      else:
        print('중복 제거', p.title)
  return array_rss

In [4]:
list_articles = crawl_rss(urls)
print(list_articles)

[Crawl URL] https://rss.etnews.com/Section901.xml
[Crawl URL] https://rss.etnews.com/Section902.xml
중복 제거 [2024 美 대선] 붉게 물든 美 지도…트럼프 백악관 입성 '눈앞'
중복 제거 [美 트럼프 시대]그가 돌아왔다..경합주 잡고 '승리 선언'
중복 제거 [美 47대 대통령 트럼프] 글로벌 증시 휘청···中 하락하고 日 상승하고
중복 제거 [美 47대 대통령 트럼프]국민 과반 득표, '완전한 승리'로 집권 2기 시작
중복 제거 [美 트럼프 시대]트럼프 승리에 환율 1400원 육박, 비트코인 7만 5000달러 경신
중복 제거 [美 47대 대통령 트럼프] 트럼프 정권 2기 백악관·내각 구성에 '주목'
중복 제거 [美 트럼프 시대]미국 중심주의·중국 배제 기조 강화…한국도 불확실성 커져
[Crawl URL] https://rss.etnews.com/Section903.xml
중복 제거 트럼프 선거인단 과반 넘어 277표 확보… 폭스 "트럼프 당선"
[Crawl URL] https://rss.etnews.com/Section904.xml
중복 제거 [2024 美 대선] 붉게 물든 美 지도…트럼프 백악관 입성 '눈앞'
중복 제거 [美 트럼프 시대]그가 돌아왔다..경합주 잡고 '승리 선언'
중복 제거 [美 47대 대통령 트럼프]국민 과반 득표, '완전한 승리'로 집권 2기 시작
중복 제거 [美 트럼프 시대]트럼프 승리에 환율 1400원 육박, 비트코인 7만 5000달러 경신
중복 제거 [美 47대 대통령 트럼프] 트럼프 정권 2기 백악관·내각 구성에 '주목'
중복 제거 [美 트럼프 시대]미국 중심주의·중국 배제 기조 강화…한국도 불확실성 커져
중복 제거 “플랫폼 자사우대 규제 신중해야”…규제와 혁신의 균형 세미나
중복 제거 내년 2조3413억원 규모 기초연구사업 본격 착수…신규과제 대폭 확대
중복 제거 730억 과징금 맞은 메타, 불복 소송만 4건 제기…“법원서 승부 갈리는데”
중복 제

## 단계 2: list에 존재하는 모든 기사들의 link를 하나씩 돌아다니며 본문 title과 text 추출

In [5]:
def crawl_article(url, language='ko'):
  print('[Crawl article]', url)
  a = Article(url, language=language)
  a.download()
  a.parse()
  return a.title, a.text

In [6]:
for article in list_articles:
  _, text = crawl_article(article['link'])
  article['text'] = text

print(list_articles[0])

[Crawl article] https://www.etnews.com/20241106000411
[Crawl article] https://www.etnews.com/20241106000409
[Crawl article] https://www.etnews.com/20241106000402
[Crawl article] https://www.etnews.com/20241106000396
[Crawl article] https://www.etnews.com/20241106000392
[Crawl article] https://www.etnews.com/20241106000389
[Crawl article] https://www.etnews.com/20241106000388
[Crawl article] https://www.etnews.com/20241106000386
[Crawl article] https://www.etnews.com/20241106000280
[Crawl article] https://www.etnews.com/20241106000227
[Crawl article] https://www.etnews.com/20241106000381
[Crawl article] https://www.etnews.com/20241106000377
[Crawl article] https://www.etnews.com/20241106000205
[Crawl article] https://www.etnews.com/20241106000166
[Crawl article] https://www.etnews.com/20241106000330
[Crawl article] https://www.etnews.com/20241106000206
[Crawl article] https://www.etnews.com/20241106000187
[Crawl article] https://www.etnews.com/20241106000159
[Crawl article] https://www.

## 단계 3: 긁어온 본문 text에서 명사 추출(키워드, 빈도수)

In [7]:
def get_keywords(text, nkeywords=10):
  list_keywords = []
  spliter = Okt()
  nouns = spliter.nouns(text)
  count = Counter(nouns)

  for n, c in count.most_common(nkeywords):
    item = {'keyword':n, 'count':c}
    list_keywords.append(item)

  return list_keywords

In [8]:
for article in list_articles:
  keywords = get_keywords(article['text'])
  article['keywords'] = keywords

print(list_articles[0])

{'title': "[2024 美 대선] 붉게 물든 美 지도…트럼프 백악관 입성 '눈앞'", 'link': 'https://www.etnews.com/20241106000411', 'text': "47대 미국 대통령 선거 개표가 순차적으로 진행 중인 가운데 공화당 대통령 후보인 도널드 트럼프 전 미국 대통령에 우 '붉은 신기루' 현상이 나타다. 다만 아직 개표 중반이고 사전투표율이 높아 시간이 지나면 민주당 대통령 후보인 카멀라 해리스 부통령에 쏠리는 '푸른 전환'이 나타날 수 있다는 분석도 있다.\n\n\n\n6일(이하 한국시간) 미국 대통령 선거 개표가 진행 중에 있다. AP통신에 따르면 오후 3시 15분 기준으로 트럼프 후보는 선거인단 247명을 확보하며 214명을 확보한 해리스 후보를 크게 앞서고 있다. 트럼프 후보는 최대 승부처로 꼽히는 펜실베이니아와 더불어 경합주인 노스캐롤라이나, 조지아에서 승리를 했고 다른 경합 지역인 위스콘신, 미시간에서도 앞서고 있다. 이에 트럼프 후보의 백악관 재입성을 예측하는 분석이 나오고 있다.\n\n\n\n아직까지 선거 결과를 예단하기에는 이르다는 목소리도 있다. 각 주별 개표 시간과 방식이 제각각이여서 선거 결과가 언제, 어떻게 나올지 유동적이라는 것이다.\n\n\n\n특히 우편투표를 포함한 사전투표가 늘어난 것도 변수로 작용할 가능성이 있다. 이번 대선 사전투표에 참여한 유권자는 8600만명으로 전체 유권자의 1/3 이상이다. 2016년 대선(4724만명) 사전투표율을 크게 상회한다. 통상 사전투표율이 높으면 민주당에 유리하다고 본다. 지난 대선 당시 본투표 개표 때는 트럼프 후보가 사전투표가 개표되면서 우위가 사라지는 현상이 발생한 바 있다.\n\n\n\n사전투표가 개표 시작된다면 된다면 해리스 후보에게 표가 쏠리는 '푸른 전환'이 나타날 수 있다는 것이다. 해리스 후보가 근소한 차이로 트럼프 후보를 추격한다면 역전이 가능할 것이라는 시나리오다. 다만 민주당 뿐만 아니라 공화당에서도 사전투표 참여를 독려한만큼 어느

## 단계 4: 검색어를 입력받아서 그 검색어를 가지고 있는 문서 출력

In [9]:
print('질의어를 입력하세요: ')
query = input()

질의어를 입력하세요: 
대선


In [10]:
def search_articles(query, list_keywords):
  nWords = 0
  for kw in list_keywords:
    if query == kw['keyword']:
      nWords += kw['count']
  return nWords

In [11]:
array_search = []

for article in list_articles:
  nQuery = search_articles(query, article['keywords'])
  if nQuery != 0:
    array_search.append([nQuery, article['title'], article['link']])

array_search.sort(key=lambda x: x[0], reverse=True)

print('-------------------')
for p in array_search:
  print(p[1])
  print(p[2], '[TF]', p[0])
print('-------------------')

-------------------
[美 트럼프 시대] 심판론에 샤이 트럼프 결집까지…더 강력해진 스트롱맨
https://www.etnews.com/20241106000410 [TF] 7
막오른 `미국의 선택`...해리스-트럼프 초접전
https://www.etnews.com/20241105000263 [TF] 6
[美 트럼프 시대]그가 돌아왔다..경합주 잡고 '승리 선언'
https://www.etnews.com/20241106000409 [TF] 5
트럼프, 승리 선언하나… “지지자 연설 위해 팜비치로 이동 중”
https://www.etnews.com/20241106000330 [TF] 5
트럼프 선거인단 과반 넘어 277표 확보… 폭스 "트럼프 당선"
https://www.etnews.com/20241106000246 [TF] 2
-------------------
